In [ ]:
import kerastuner as kt
import keras
from sklearn import ensemble
from sklearn import datasets
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
import pandas as pd

data_dir = 'D:\FINKI\8_dps\Project\DATA\MODELS_DATA\\base_models_all_dft_coefficients'
train = pd.read_csv(f'{data_dir}\\segments_10s_train.csv')
test = pd.read_csv(f'{data_dir}\\segments_10s_test.csv')

y_train = train.pop['episode']
y_test = test.pop['episode']

def build_model(hp):

  model = keras.models.Sequential()

  num_layers = hp.Int('num_layers', 1, 4, step = 1)
  num_nodes_first_layer = hp.Int('num_nodes_first_layer', 20, 200)
  shrinkage_factor = hp.float('shrinkage_factor', 0.7, 0.3)
  learning_rate = hp.Float('learning_rate', 0.001, 0.5)
  decay = hp.Float('decay', 0.001, 0.1)
  optimizer = hp.Choice('optimizer', ['Adam', 'Nadam', 'RMSProp', 'Adagrad'])

  model.add(keras.layers.Dense(units = layer_dict['units'],
                      input_shape = (self.split_sets['X_train'].shape[1],),
                      activation = layer_dict['activation'],
                      kernel_initializer = self.hypermethods['initializer'],
                      kernel_regularizer = self.hypermethods['regularizer']))

  for layer_iter in range(num_layers):
      model.add(keras.layers.Dense(units = layer_dict['units'],
                      input_shape = (self.split_sets['X_train'].shape[1],),
                      activation = layer_dict['activation'],
                      kernel_initializer = self.hypermethods['initializer'],
                      kernel_regularizer = self.hypermethods['regularizer']))


  model_type = hp.Choice('model_type', ['random_forest', 'ridge'])
  if model_type == 'random_forest':
    model = ensemble.RandomForestClassifier(
        n_estimators=hp.Int('n_estimators', 10, 50, step=10),
        max_depth=hp.Int('max_depth', 3, 10))
  else:
    model = linear_model.RidgeClassifier(
        alpha=hp.Float('alpha', 1e-3, 1, sampling='log'))
  return model

tuner = kt.tuners.Sklearn(
    oracle=kt.oracles.BayesianOptimization(
        objective=kt.Objective('score', 'max'),
        max_trials=10),
    hypermodel=build_model,
    scoring=metrics.make_scorer(metrics.accuracy_score),
    cv=model_selection.StratifiedKFold(5),
    directory='.',
    project_name='my_project')

X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.2)

tuner.search(X_train, y_train)

best_model = tuner.get_best_models(num_models=1)[0]